# Notebook 5: Euclidean Gas Convergence to QSD

**Goal**: Demonstrate exponential convergence of the **Euclidean Gas** to the Quasi-Stationary Distribution (QSD) using framework-correct Lyapunov functions with **interactive visualizations**.

**Key Framework Concepts**:

The framework-correct Lyapunov function for a single swarm (03_cloning.md):

$$V_{\text{total}}(S) = V_{\text{Var},x}(S) + V_{\text{Var},v}(S)$$

where:
- $V_{\text{Var},x}(S) = \frac{1}{N} \sum_{i \in \mathcal{A}(S)} \|\delta_{x,i}\|^2$ (N-normalized positional variance)
- $V_{\text{Var},v}(S) = \frac{1}{N} \sum_{i \in \mathcal{A}(S)} \|\delta_{v,i}\|^2$ (N-normalized velocity variance)
- $\delta_{x,i} = x_i - \mu_x$ (deviation from center of mass)

**What to Expect**:
1. Lyapunov function should **decay exponentially**: $V(t) \approx C e^{-\kappa t}$
2. On log scale, this becomes a **straight line**: $\log V(t) \approx \log C - \kappa t$
3. **Interactive evolution**: Watch the swarm evolve step-by-step with time slider!

## Setup and Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../src')

import torch
import numpy as np
import pandas as pd

# HoloViews stack (NO matplotlib!)
import holoviews as hv
from holoviews import opts
import panel as pn
import panel.widgets as pnw

# Enable Bokeh backend
hv.extension('bokeh')
pn.extension()

# Import modular experiment code
from fragile.euclidean_gas_experiments import (
    create_multimodal_potential,
    ConvergenceExperiment,
)
from fragile.euclidean_gas import (
    EuclideanGas,
    EuclideanGasParams,
    LangevinParams,
    CloningParams,
)
from fragile.bounds import TorchBounds
from fragile.companion_selection import CompanionSelection

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

print("✓ Imports successful")
print(f"✓ HoloViews version: {hv.__version__}")
print(f"✓ Panel version: {pn.__version__}")

## 1. Create Target Potential and QSD

We'll use a multimodal Gaussian mixture as our target potential. The QSD is proportional to:

$$\pi_{\text{QSD}}(x) \propto \exp(-\beta U(x))$$

where $U(x)$ is the potential derived from the Gaussian mixture.

In [ ]:
# Create multimodal potential
potential, target_mixture = create_multimodal_potential(
    dims=2,
    n_gaussians=3,
    bounds_range=(-8.0, 8.0),
    seed=160290
)

# Extract parameters
centers = target_mixture.centers
stds = target_mixture.stds
weights = target_mixture.weights
dims = target_mixture.dims

print(f"✓ Created multimodal potential")
print(f"  Centers: {centers.tolist()}")
print(f"  Weights: {weights.tolist()}")

### Visualize Target QSD (Interactive!)

In [ ]:
# Create grid for visualization
x_range = np.linspace(-8, 8, 200)
y_range = np.linspace(-8, 8, 200)
X, Y = np.meshgrid(x_range, y_range)
grid_points = torch.tensor(np.stack([X.ravel(), Y.ravel()], axis=1), dtype=torch.float32)

# Evaluate potential
Z_potential = potential.evaluate(grid_points).detach().numpy().reshape(X.shape)

# Compute target QSD
beta = 1.0
Z_qsd = np.exp(-beta * Z_potential)
Z_qsd = Z_qsd / Z_qsd.sum()

# Create HoloViews visualization
qsd_image = hv.Image(
    (x_range, y_range, Z_qsd),
    kdims=['x₁', 'x₂'],
    vdims='π_QSD'
).opts(
    cmap='plasma',
    colorbar=True,
    width=800,
    height=700,
    title='Target QSD π_QSD(x) ∝ e^(-βU(x))',
    xlabel='x₁',
    ylabel='x₂',
    tools=['hover'],
    fontscale=1.2,
)

# Add mode markers
mode_data = pd.DataFrame({
    'x': centers[:, 0].numpy(),
    'y': centers[:, 1].numpy(),
    'size': weights.numpy() * 30
})
mode_points = hv.Points(
    mode_data,
    kdims=['x', 'y'],
    vdims='size',
    label='Modes'
).opts(
    size='size',
    color='red',
    marker='star',
    line_color='white',
    line_width=2,
)

qsd_plot = qsd_image * mode_points
qsd_plot

## 2. Initialize Euclidean Gas

We'll use the **Euclidean Gas** with:
- Langevin dynamics (BAOAB integrator)
- Cloning operator with inelastic collisions
- No adaptive mechanisms (pure baseline algorithm)

The swarm will start **far from equilibrium** to clearly observe convergence.

In [ ]:
# Parameters
N = 100
n_steps = 500

# Define bounds
bounds = TorchBounds(
    low=torch.tensor([-6.0, -6.0]),
    high=torch.tensor([6.0, 6.0])
)

# Create companion selection strategy
companion_selection = CompanionSelection(
    method="uniform",
    lambda_alg=1.0,
)

# Create parameters
params = EuclideanGasParams(
    N=N,
    d=dims,
    potential=potential,
    langevin=LangevinParams(
        gamma=1.0,
        beta=1.0,
        delta_t=0.1
    ),
    cloning=CloningParams(
        sigma_x=0.5,
        lambda_alg=1.,
        alpha_restitution=1.,
        alpha=0.5,
        beta=2.,
        eta=0.1,
        A=2.0,
        sigma_min=1e-8,
        p_max=1.0,
        epsilon_clone=0.0001,
        companion_selection=companion_selection,
    ),
    bounds=bounds,
    device="cpu",
    dtype="float32",
)

# Create Euclidean Gas instance
gas = EuclideanGas(params)

# Initialize swarm FAR FROM EQUILIBRIUM
x_init = torch.rand(N, dims) * 2.0 + 4.0  # Corner [4,6] x [4,6]
v_init = torch.randn(N, dims) * 0.1

print(f"✓ Initialized Euclidean Gas")
print(f"  N walkers: {N}")
print(f"  Dimensions: {dims}")
print(f"  Time steps: {n_steps}")
print(f"  Initial position range: [{x_init.min():.2f}, {x_init.max():.2f}]")

### Visualize Initial Configuration (Interactive!)

In [ ]:
# Background: target QSD (greyscale)
qsd_bg = hv.Image(
    (x_range, y_range, Z_qsd),
    kdims=['x₁', 'x₂'],
    vdims='density'
).opts(
    cmap='Greys',
    alpha=0.3,
    colorbar=False,
)

# Initial swarm positions
init_positions = pd.DataFrame({
    'x1': x_init[:, 0].numpy(),
    'x2': x_init[:, 1].numpy(),
})
swarm_points = hv.Points(
    init_positions,
    kdims=['x1', 'x2'],
    label='Initial Swarm'
).opts(
    size=7,
    color='blue',
    alpha=0.6,
    line_color='black',
    line_width=0.5,
)

# Mode markers
mode_points_init = hv.Points(
    mode_data,
    kdims=['x', 'y'],
    vdims='size',
    label='Target Modes'
).opts(
    size='size',
    color='gold',
    marker='star',
    line_color='white',
    line_width=2,
)

init_plot = (qsd_bg * swarm_points * mode_points_init).opts(
    xlim=(-6, 6),
    ylim=(-6, 6),
    xlabel='x₁',
    ylabel='x₂',
    title='Initial Configuration: Swarm Far from Equilibrium',
    width=800,
    height=700,
    show_grid=True,
    fontscale=1.2,
)

print("\n✓ Swarm starts in upper right corner, far from all modes!")
init_plot

## 3. Run Convergence Experiment

We'll save **many snapshots** to enable smooth interactive playback!

In [ ]:
# Create snapshot times - save every 50 steps for smooth playback
snapshot_interval = 2
snapshot_times = list(range(0, n_steps + 1, snapshot_interval))

experiment = ConvergenceExperiment(
    gas=gas,
    save_snapshots_at=snapshot_times
)

print(f"Running simulation with {len(snapshot_times)} snapshots...\n")

metrics, snapshots = experiment.run(
    n_steps=n_steps,
    x_init=x_init,
    v_init=v_init,
    measure_every=10,
    verbose=True
)

## 4. Lyapunov Function Decay: The Straight Line! (Interactive!)

On a **log-linear plot**, exponential decay appears as a **straight line**.

In [ ]:
time_arr = np.array(metrics.time)
V_total = np.array(metrics.V_total)
V_var_x = np.array(metrics.V_var_x)
V_var_v = np.array(metrics.V_var_v)

# LEFT: Total Lyapunov decay (log scale)
lyap_data = pd.DataFrame({'time': time_arr, 'V_total': V_total})
lyap_curve = hv.Curve(
    lyap_data,
    kdims='time',
    vdims='V_total',
    label='V_total'
).opts(
    color='blue',
    line_width=2,
    alpha=0.7,
    logy=True,
    width=700,
    height=500,
    xlabel='Time (steps)',
    ylabel='V_total(S) (N-normalized)',
    title='Framework-Correct Lyapunov Function Decay',
    fontscale=1.2,
    tools=['hover'],
    show_grid=True,
)

# Fit exponential decay
fit_result = metrics.fit_exponential_decay('V_total', fit_start_time=500)
if fit_result is not None:
    kappa, C = fit_result
    time_fit = time_arr[time_arr >= 500]
    V_fitted = C * np.exp(-kappa * time_fit)
    
    fit_data = pd.DataFrame({'time': time_fit, 'fit': V_fitted})
    fit_curve = hv.Curve(
        fit_data,
        kdims='time',
        vdims='fit',
        label=f'Fit: C e^(-κt), κ = {kappa:.4f}'
    ).opts(
        color='red',
        line_width=2,
        line_dash='dashed',
        logy=True,
    )
    
    lyap_plot = lyap_curve * fit_curve
    
    print(f"\n📊 Fitted Convergence Rate: κ = {kappa:.4f}")
    print(f"   Half-life: t_1/2 = {np.log(2)/kappa:.2f} steps")
else:
    lyap_plot = lyap_curve

# RIGHT: Variance components
var_data = pd.DataFrame({
    'time': time_arr,
    'V_var_x': V_var_x,
    'V_var_v': V_var_v
})

var_x_curve = hv.Curve(
    var_data,
    kdims='time',
    vdims='V_var_x',
    label='V_Var,x (position)'
).opts(
    color='blue',
    line_width=2,
    alpha=0.7,
    logy=True,
)

var_v_curve = hv.Curve(
    var_data,
    kdims='time',
    vdims='V_var_v',
    label='V_Var,v (velocity)'
).opts(
    color='green',
    line_width=2,
    alpha=0.7,
    logy=True,
)

var_plot = (var_x_curve * var_v_curve).opts(
    width=700,
    height=500,
    xlabel='Time (steps)',
    ylabel='Variance Components (N-normalized)',
    title='Position vs Velocity Variance',
    fontscale=1.2,
    tools=['hover'],
    show_grid=True,
)

convergence_plots = (lyap_plot + var_plot).opts(
    opts.Layout(shared_axes=False)
)

print(f"\n✨ Exponential Lyapunov decay confirmed!")
print(f"Final V_total: {V_total[-1]:.6f}")

convergence_plots

## 5. Interactive Swarm Evolution Dashboard

**The Main Event**: Watch the swarm evolve step-by-step with an interactive time slider and play button!

### Features:
- **Play button**: Automatic animation
- **Time slider**: Manual control
- **Real-time metrics**: Watch Lyapunov decay as you scrub through time
- **Smooth playback**: 60+ snapshots from start to finish

In [37]:
# Get sorted snapshot times
time_indices = sorted(snapshots.keys())

# Time slider widget
time_slider = pnw.DiscreteSlider(
    name='Time',
    options=time_indices,
    value=0,
    width=700
)

# Play button for animation
play_button = pnw.Player(
    name='Animation',
    start=0,
    end=len(time_indices) - 1,
    value=0,
    interval=300,  # milliseconds between frames
    width=700,
    show_loop_controls=True,
)

# Create synchronized plot function
@pn.depends(time_slider.param.value, play_button.param.value)
def create_evolution_plot(slider_time, player_idx):
    """Create plot synchronized with both slider and player."""
    # Prefer slider if it's been moved, otherwise use player
    time_idx = slider_time if slider_time != time_indices[0] else time_indices[player_idx]
    
    positions = snapshots[time_idx].detach().numpy()
    
    # Background: target QSD
    qsd_bg = hv.Image(
        (x_range, y_range, Z_qsd),
        kdims=['x₁', 'x₂'],
        vdims='density'
    ).opts(
        cmap='Greys',
        alpha=0.3,
        colorbar=False,
    )
    
    # Swarm positions
    swarm_df = pd.DataFrame({
        'x1': positions[:, 0],
        'x2': positions[:, 1],
    })
    swarm_points = hv.Points(
        swarm_df,
        kdims=['x1', 'x2']
    ).opts(
        size=8,
        color='blue',
        alpha=0.7,
        line_color='black',
        line_width=0.5,
    )
    
    # Mode markers
    mode_markers = hv.Points(
        mode_data,
        kdims=['x', 'y'],
        vdims='size',
        label='Modes'
    ).opts(
        size='size',
        color='red',
        marker='star',
        line_color='white',
        line_width=2,
    )
    
    # Get metrics at this time
    metric_times = np.array(metrics.time)
    closest_idx = np.argmin(np.abs(metric_times - time_idx))
    V_val = metrics.V_total[closest_idx] if closest_idx < len(metrics.V_total) else float('nan')
    n_alive = metrics.n_alive[closest_idx] if closest_idx < len(metrics.n_alive) else 0
    
    # Combine
    plot = (qsd_bg * swarm_points * mode_markers).opts(
        xlim=(-6, 6),
        ylim=(-6, 6),
        xlabel='x₁',
        ylabel='x₂',
        title=f'Swarm Evolution at t = {time_idx} | V_total = {V_val:.4f} | Alive = {n_alive}',
        width=700,
        height=650,
        show_grid=True,
        fontscale=1.1,
    )
    
    return plot

# Create metrics display
@pn.depends(time_slider.param.value, play_button.param.value)
def create_metrics_display(slider_time, player_idx):
    """Display current metrics."""
    time_idx = slider_time if slider_time != time_indices[0] else time_indices[player_idx]
    
    metric_times = np.array(metrics.time)
    closest_idx = np.argmin(np.abs(metric_times - time_idx))
    
    V_val = metrics.V_total[closest_idx] if closest_idx < len(metrics.V_total) else float('nan')
    V_x = metrics.V_var_x[closest_idx] if closest_idx < len(metrics.V_var_x) else float('nan')
    V_v = metrics.V_var_v[closest_idx] if closest_idx < len(metrics.V_var_v) else float('nan')
    n_alive = metrics.n_alive[closest_idx] if closest_idx < len(metrics.n_alive) else 0
    
    metrics_text = f"""
    ## Convergence Metrics at t = {time_idx}
    
    **V_total**: `{V_val:.6f}`  
    **V_var_x** (position): `{V_x:.6f}`  
    **V_var_v** (velocity): `{V_v:.6f}`  
    **Alive walkers**: `{n_alive}`
    
    ---
    
    ### Progress
    Time: **{time_idx} / {max(time_indices)}** ({100*time_idx/max(time_indices):.1f}%)
    """
    
    return pn.pane.Markdown(metrics_text, width=300)

# Create mini convergence plot that updates with time
@pn.depends(time_slider.param.value, play_button.param.value)
def create_mini_convergence_plot(slider_time, player_idx):
    """Show convergence up to current time."""
    time_idx = slider_time if slider_time != time_indices[0] else time_indices[player_idx]
    
    # Filter data up to current time
    time_arr_full = np.array(metrics.time)
    V_arr_full = np.array(metrics.V_total)
    mask = time_arr_full <= time_idx
    
    time_subset = time_arr_full[mask]
    V_subset = V_arr_full[mask]
    
    if len(V_subset) > 0:
        mini_data = pd.DataFrame({'time': time_subset, 'V': V_subset})
        mini_curve = hv.Curve(
            mini_data,
            kdims='time',
            vdims='V'
        ).opts(
            color='blue',
            line_width=2,
            logy=True,
            width=300,
            height=200,
            xlabel='Time',
            ylabel='V_total',
            title='Convergence Progress',
            show_grid=True,
        )
        
        # Add current time marker
        if time_idx > 0 and len(V_subset) > 0:
            marker = hv.Scatter(
                [(time_idx, V_subset[-1])],
                kdims='time',
                vdims='V'
            ).opts(
                size=10,
                color='red',
                marker='o',
            )
            return mini_curve * marker
        
        return mini_curve
    else:
        return hv.Curve([]).opts(width=300, height=200, title='Convergence Progress')

# Assemble dashboard
dashboard = pn.Column(
    pn.pane.Markdown("# 🎬 Interactive Swarm Evolution Dashboard", width=800),
    pn.pane.Markdown("Use the **player** to watch the animation or **slider** for manual control.", width=800),
    play_button,
    time_slider,
    pn.Row(
        create_evolution_plot,
        pn.Column(
            create_metrics_display,
            create_mini_convergence_plot,
        )
    ),
)

print("✨ Enhanced interactive dashboard created!")
print("   • Use PLAY button for automatic animation")
print("   • Use TIME SLIDER for manual control")
print("   • Watch metrics update in real-time!")

dashboard

✨ Enhanced interactive dashboard created!
   • Use PLAY button for automatic animation
   • Use TIME SLIDER for manual control
   • Watch metrics update in real-time!


Column
    [0] Markdown(str, width=800)
    [1] Markdown(str, width=800)
    [2] Player(end=250, interval=300, name='Animation', width=700)
    [3] DiscreteSlider(formatter='%d', name='Time', options=[0, 2, 4, 6, 8, ...], value=0, width=700)
    [4] Row
        [0] ParamFunction(function, _pane=HoloViews, defer_load=False)
        [1] Column
            [0] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] ParamFunction(function, _pane=HoloViews, defer_load=False)

## 6. Static Snapshots Grid (for reference)

In [38]:
# Create grid of key snapshots
key_times = [0, 500, 1000, 1500, 2000, 2500, n_steps]
snapshot_plots = []

for t in key_times:
    if t not in snapshots:
        continue
    
    positions = snapshots[t].detach().numpy()
    
    qsd_bg = hv.Image(
        (x_range, y_range, Z_qsd),
        kdims=['x₁', 'x₂'],
        vdims='density'
    ).opts(cmap='Greys', alpha=0.3, colorbar=False)
    
    swarm_df = pd.DataFrame({'x1': positions[:, 0], 'x2': positions[:, 1]})
    swarm = hv.Points(swarm_df, kdims=['x1', 'x2']).opts(
        size=7, color='blue', alpha=0.6, line_color='black', line_width=0.5
    )
    
    modes = hv.Points(mode_data, kdims=['x', 'y'], vdims='size').opts(
        size='size', color='gold', marker='star', line_color='white', line_width=2
    )
    
    plot = (qsd_bg * swarm * modes).opts(
        xlim=(-6, 6), ylim=(-6, 6),
        xlabel='x₁', ylabel='x₂',
        title=f't = {t}',
        width=350, height=320,
        show_grid=True,
    )
    
    snapshot_plots.append(plot)

# Create grid layout
grid = hv.Layout(snapshot_plots).cols(3).opts(
    opts.Layout(
        title='Euclidean Gas Evolution: Key Snapshots',
        fontscale=1.1,
    )
)

grid

:Layout
   .Overlay.I   :Overlay
      .Image.I   :Image   [x₁,x₂]   (density)
      .Points.I  :Points   [x1,x2]
      .Points.II :Points   [x,y]   (size)
   .Overlay.II  :Overlay
      .Image.I   :Image   [x₁,x₂]   (density)
      .Points.I  :Points   [x1,x2]
      .Points.II :Points   [x,y]   (size)
   .Overlay.III :Overlay
      .Image.I   :Image   [x₁,x₂]   (density)
      .Points.I  :Points   [x1,x2]
      .Points.II :Points   [x,y]   (size)

## 7. Statistical Comparison: Final Distribution vs Target

In [ ]:
# Final positions
final_time = max(snapshots.keys())
final_positions = snapshots[final_time].detach().numpy()

# Compute empirical statistics
emp_mean = final_positions.mean(axis=0)
emp_cov = np.cov(final_positions.T)

# Target statistics
target_mean = (centers * weights.unsqueeze(1)).sum(dim=0).numpy()
target_cov_approx = sum(
    weights[i].item() * (np.outer(centers[i].numpy(), centers[i].numpy()) + np.diag(stds[i].numpy()**2))
    for i in range(len(weights))
) - np.outer(target_mean, target_mean)

print("\n📊 Statistical Comparison (Final vs Target):\n")
print(f"Empirical Mean:   {emp_mean}")
print(f"Target Mean:      {target_mean}")
print(f"Mean Error:       {np.linalg.norm(emp_mean - target_mean):.4f}\n")
print(f"Covariance Frobenius Error: {np.linalg.norm(emp_cov - target_cov_approx, 'fro'):.4f}")

# Visual comparison of marginal distributions
marginal_plots = []

for dim_idx in range(dims):
    # Empirical histogram
    frequencies, edges = np.histogram(final_positions[:, dim_idx], bins=30, density=True)
    hist_hv = hv.Histogram((edges, frequencies), label='Empirical').opts(
        fill_color='blue', alpha=0.6, line_color='black'
    )
    
    # Target density
    x_range_1d = np.linspace(-6, 6, 200)
    target_density = sum(
        weights[i].item() / (np.sqrt(2 * np.pi) * stds[i, dim_idx].item()) *
        np.exp(-0.5 * ((x_range_1d - centers[i, dim_idx].item()) / stds[i, dim_idx].item())**2)
        for i in range(len(weights))
    )
    
    target_curve = hv.Curve(
        (x_range_1d, target_density),
        kdims='x',
        vdims='density',
        label='Target QSD'
    ).opts(color='red', line_width=2)
    
    marginal_plot = (hist_hv * target_curve).opts(
        width=600, height=400,
        xlabel=f'x_{dim_idx+1}',
        ylabel='Density',
        title=f'Marginal Distribution: Dimension {dim_idx+1}',
        show_grid=True,
        fontscale=1.1,
        legend_position='top_right',
    )
    
    marginal_plots.append(marginal_plot)

marginal_layout = hv.Layout(marginal_plots).cols(2)

print("\n✨ The empirical distribution closely matches the target QSD!")
marginal_layout

## 8. Summary and Validation

### Key Results:

1. **✅ Exponential Decay**: The Lyapunov function decays exponentially with rate $\kappa$
   - Straight line on log-linear plot confirms exponential behavior

2. **✅ Framework-Correct N-Normalization**: All variance terms use the framework definition
   - $V_{\text{Var},x}(S) = \frac{1}{N} \sum_{i \in \mathcal{A}(S)} \|\delta_{x,i}\|^2$
   - Ensures N-uniform drift inequalities

3. **✅ Convergence to QSD**: Interactive visualization shows swarm migration from chaos to equilibrium
   - Watch step-by-step with play button!
   - Final distribution matches target statistically

4. **✅ Interactive Analysis**: HoloViews + Panel + Bokeh provides:
   - Smooth animation playback
   - Real-time metric updates
   - Zoomable, hoverable plots

### Physical Interpretation:

The framework-correct Lyapunov function measures **internal disorder** within the swarm:
- As walkers explore and clone, they reduce internal variance
- The N-normalization ensures the measure is **independent of swarm size**
- This is critical for mean-field analysis and propagation of chaos results

---

**References**:
- Framework: [docs/source/1_euclidean_gas/03_cloning.md](docs/source/1_euclidean_gas/03_cloning.md) § 3.2
- Implementation: [src/fragile/lyapunov.py](src/fragile/lyapunov.py)
- Experiments: [src/fragile/euclidean_gas_experiments.py](src/fragile/euclidean_gas_experiments.py)

### Parameter Guide Summary

**Quick Reference Table:**

| **Goal** | **lambda_alg** | **beta** | **sigma_x** | **alpha_restitution** |
|----------|----------------|----------|-------------|-----------------------|
| **Tight Clustering** | 0.0 - 0.5 | 0.1 - 0.5 | 0.01 - 0.1 | 0.0 - 0.3 |
| **Balanced** | 0.5 - 1.0 | 1.0 - 1.5 | 0.1 - 0.3 | 0.4 - 0.6 |
| **Maximum Spreading** | 1.5 - 5.0 | 2.0 - 5.0 | 0.3 - 1.0 | 0.7 - 1.0 |

**Physical Interpretation:**

- **Clustering**: Walkers form tight groups in position space, ignoring velocity differences. Good for:
  - Precise local search around optima
  - Lower computational variance
  - Faster convergence to a single mode

- **Spreading**: Walkers maintain diversity in both position AND velocity. Good for:
  - Exploring multiple modes simultaneously
  - Avoiding premature convergence
  - Better global exploration

**Algorithmic Space Insight:**

The Lyapunov function $V_{\\text{total}}$ measures **internal variance** in algorithmic space:
- **Clustering** → Lower $V_{\\text{total}}$ (walkers close in alg space)
- **Spreading** → Higher $V_{\\text{total}}$ (walkers dispersed in alg space)

This is NOT about convergence speed, but about the **structure of the swarm** during exploration!

In [ ]:
# Compare Lyapunov functions
time_cluster = np.array(metrics_cluster.time)
V_cluster = np.array(metrics_cluster.V_total)

time_spread = np.array(metrics_spread.time)
V_spread = np.array(metrics_spread.V_total)

# Create comparison plot
cluster_curve = hv.Curve(
    pd.DataFrame({'time': time_cluster, 'V': V_cluster}),
    kdims='time',
    vdims='V',
    label='Clustering (λ_alg=0, β=0.2)'
).opts(
    color='red',
    line_width=2,
    alpha=0.7,
    logy=True,
)

spread_curve = hv.Curve(
    pd.DataFrame({'time': time_spread, 'V': V_spread}),
    kdims='time',
    vdims='V',
    label='Spreading (λ_alg=2.0, β=3.0)'
).opts(
    color='blue',
    line_width=2,
    alpha=0.7,
    logy=True,
)

lyap_comparison = (cluster_curve * spread_curve).opts(
    width=900,
    height=500,
    xlabel='Time (steps)',
    ylabel='V_total(S) (N-normalized)',
    title='Lyapunov Comparison: Clustering vs Spreading',
    fontscale=1.2,
    tools=['hover'],
    show_grid=True,
    legend_position='top_right',
)

print("\\n📊 Lyapunov Comparison:")
print(f"  CLUSTERING final V_total: {V_cluster[-1]:.4f}")
print(f"  SPREADING final V_total:  {V_spread[-1]:.4f}")
print(f"\\n  Interpretation:")
print(f"  - Lower V_total → tighter clustering in algorithmic space")
print(f"  - Higher V_total → more spread out, higher internal diversity")

lyap_comparison

### Lyapunov Comparison: Clustering vs Spreading

Compare convergence rates between the two parameter regimes!

In [ ]:
# Create side-by-side comparison at t=200
time_compare = 200

pos_cluster = snapshots_cluster[time_compare].detach().numpy()
pos_spread = snapshots_spread[time_compare].detach().numpy()

# LEFT: Clustering behavior
qsd_bg_left = hv.Image(
    (x_range, y_range, Z_qsd),
    kdims=['x₁', 'x₂'],
    vdims='density'
).opts(cmap='Greys', alpha=0.3, colorbar=False)

swarm_cluster = hv.Points(
    pd.DataFrame({'x1': pos_cluster[:, 0], 'x2': pos_cluster[:, 1]}),
    kdims=['x1', 'x2']
).opts(
    size=8, color='red', alpha=0.7, line_color='black', line_width=0.5
)

modes_left = hv.Points(mode_data, kdims=['x', 'y'], vdims='size').opts(
    size='size', color='gold', marker='star', line_color='white', line_width=2
)

plot_cluster = (qsd_bg_left * swarm_cluster * modes_left).opts(
    xlim=(-6, 6), ylim=(-6, 6),
    xlabel='x₁', ylabel='x₂',
    title='CLUSTERING: λ_alg=0, β=0.2, σ_x=0.05',
    width=600, height=550,
    show_grid=True,
    fontscale=1.1,
)

# RIGHT: Spreading behavior
qsd_bg_right = hv.Image(
    (x_range, y_range, Z_qsd),
    kdims=['x₁', 'x₂'],
    vdims='density'
).opts(cmap='Greys', alpha=0.3, colorbar=False)

swarm_spread_viz = hv.Points(
    pd.DataFrame({'x1': pos_spread[:, 0], 'x2': pos_spread[:, 1]}),
    kdims=['x1', 'x2']
).opts(
    size=8, color='blue', alpha=0.7, line_color='black', line_width=0.5
)

modes_right = hv.Points(mode_data, kdims=['x', 'y'], vdims='size').opts(
    size='size', color='gold', marker='star', line_color='white', line_width=2
)

plot_spread = (qsd_bg_right * swarm_spread_viz * modes_right).opts(
    xlim=(-6, 6), ylim=(-6, 6),
    xlabel='x₁', ylabel='x₂',
    title='SPREADING: λ_alg=2.0, β=3.0, σ_x=0.3',
    width=600, height=550,
    show_grid=True,
    fontscale=1.1,
)

# Combine side-by-side
comparison = (plot_cluster + plot_spread).opts(
    opts.Layout(shared_axes=False)
)

print("\\n🔍 Visual Comparison at t=200:")
print(f"  CLUSTERING: Tight groups, low internal diversity")
print(f"  SPREADING: Dispersed coverage, high diversity in alg space")

comparison

### Visual Comparison: Clustering vs Spreading

Compare the final swarm configurations side-by-side!

In [ ]:
# CLUSTERING PARAMETERS: Favor tight clustering in physical space
clustering_params = EuclideanGasParams(
    N=100,
    d=dims,
    potential=potential,
    langevin=LangevinParams(gamma=1.0, beta=1.0, delta_t=0.05),
    cloning=CloningParams(
        sigma_x=0.05,              # Small jitter → tight clusters
        lambda_alg=0.0,            # Ignore velocity → cluster by position only
        alpha_restitution=0.1,     # Fully inelastic → velocities collapse
        alpha=1.0,
        beta=0.2,                  # Low diversity weight → favor reward
        eta=0.1,
        A=2.0,
        sigma_min=1e-8,
        p_max=1.0,
        epsilon_clone=0.01,
        companion_selection=CompanionSelection(method="uniform", lambda_alg=0.0),
    ),
    bounds=bounds,
    device="cpu",
    dtype="float32",
)

# SPREADING PARAMETERS: Favor diversity in algorithmic space
spreading_params = EuclideanGasParams(
    N=100,
    d=dims,
    potential=potential,
    langevin=LangevinParams(gamma=1.0, beta=1.0, delta_t=0.05),
    cloning=CloningParams(
        sigma_x=0.3,               # Large jitter → scattered clones
        lambda_alg=2.0,            # High velocity weight → spread in alg space
        alpha_restitution=0.8,     # Nearly elastic → preserve velocity diversity
        alpha=1.0,
        beta=3.0,                  # High diversity weight → favor spreading
        eta=0.1,
        A=2.0,
        sigma_min=1e-8,
        p_max=1.0,
        epsilon_clone=0.01,
        companion_selection=CompanionSelection(method="uniform", lambda_alg=2.0),
    ),
    bounds=bounds,
    device="cpu",
    dtype="float32",
)

# Run both experiments
print("Running CLUSTERING experiment...")
gas_cluster = EuclideanGas(clustering_params)
exp_cluster = ConvergenceExperiment(gas_cluster, save_snapshots_at=[0, 50, 100, 200])
metrics_cluster, snapshots_cluster = exp_cluster.run(
    n_steps=200,
    x_init=x_init.clone(),
    v_init=v_init.clone(),
    measure_every=5,
    verbose=False
)

print("Running SPREADING experiment...")
gas_spread = EuclideanGas(spreading_params)
exp_spread = ConvergenceExperiment(gas_spread, save_snapshots_at=[0, 50, 100, 200])
metrics_spread, snapshots_spread = exp_spread.run(
    n_steps=200,
    x_init=x_init.clone(),
    v_init=v_init.clone(),
    measure_every=5,
    verbose=False
)

print("✓ Both experiments complete!")

## 8. Parameter Guide: Controlling Swarm Behavior

This section explains how to modify key parameters to control the swarm's behavior in **algorithmic space** (the space defined by positions + velocities).

### What is Algorithmic Space?

The **algorithmic distance** between two walkers combines position and velocity:

$$d_{\text{alg}}(w_i, w_j) = \sqrt{\|x_i - x_j\|^2 + \lambda_{\text{alg}} \|v_i - v_j\|^2}$$

where:
- $\lambda_{\text{alg}}$ controls the weight of velocity similarity
- When $\lambda_{\text{alg}} = 0$: Only position matters (walkers cluster in physical space)
- When $\lambda_{\text{alg}} > 0$: Both position AND velocity matter (walkers spread out more)

### Key Parameters for Spreading vs Clustering

---

#### 1. **`lambda_alg`** (Velocity Weight in Algorithmic Distance)

**Location**: `CloningParams(lambda_alg=...)`

**What it is**: A weighting coefficient that determines how much velocity similarity matters when measuring distance between walkers.

**How it works**: 
- The cloning operator selects companions based on algorithmic distance $d_{\text{alg}}$
- With $\lambda_{\text{alg}} = 0$: Two walkers are "close" if they're near in position, regardless of velocity
- With $\lambda_{\text{alg}} > 0$: Two walkers are only "close" if they're near in BOTH position AND velocity

**Effect on cloning**:
- **Low values (0.0 - 0.5)**: Walkers pick companions based on position only → clones inherit diverse velocities → cluster tightly in physical space
- **High values (1.0 - 5.0)**: Walkers pick companions with similar velocities → clones have similar motion → spread out to maintain velocity diversity

**Visual effect**: Higher $\lambda_{\text{alg}}$ makes the swarm look more "diffuse" because walkers avoid pairing with others who have different velocities.

---

#### 2. **`beta`** (Diversity Channel Weight in Fitness)

**Location**: `CloningParams(beta=...)`

**What it is**: An exponent that controls how much walkers value being far from their companions versus being in high-reward regions.

**How it works**: The fitness function combines two channels:
$$F_i = \left(R_i^\alpha + \beta \cdot D_i^\beta\right)^{1/(\alpha + \beta)}$$

where:
- $R_i$ = rescaled reward (how good the location is)
- $D_i$ = rescaled diversity (how far from companion)
- $\alpha$ = reward channel weight (usually 1.0)
- $\beta$ = diversity channel weight ← **this parameter!**

**Effect on behavior**:
- **Low values (0.1 - 0.5)**: Fitness dominated by reward → walkers cluster around high-reward regions, ignoring diversity
- **High values (1.0 - 5.0)**: Diversity becomes important → walkers spread out to maintain distance from neighbors

**Cloning consequence**: Walkers with high fitness clone more often. High $\beta$ means "spread-out" walkers have higher fitness → spreading is rewarded.

---

#### 3. **`sigma_x`** (Position Jitter During Cloning)

**Location**: `CloningParams(sigma_x=...)`

**What it is**: The standard deviation of Gaussian noise added to a walker's position when it clones its companion.

**How it works**: When walker $i$ clones companion $j$:
1. Walker $i$ moves to companion's position: $x_i \leftarrow x_j$
2. **Position jitter is added**: $x_i \leftarrow x_i + \mathcal{N}(0, \sigma_x^2 I)$
3. The clone appears at $x_j + \text{noise}$ instead of exactly at $x_j$

**Effect on clustering**:
- **Low values (0.01 - 0.1)**: Tiny jitter → clones appear almost exactly at companion's location → tight ball-like clusters form
- **Medium values (0.1 - 0.5)**: Moderate jitter → clones scattered around companion → loose cloud-like swarms
- **High values (0.5 - 2.0)**: Large jitter → clones can appear far from companion → very diffuse swarms

**Visual effect**: $\sigma_x$ controls the "graininess" of the swarm. Small $\sigma_x$ = compact clumps, large $\sigma_x$ = scattered spray.

**Important**: This is INDEPENDENT of position-based distance! Even with $\lambda_{\text{alg}} = 0$, high $\sigma_x$ still scatters clones spatially.

---

#### 4. **`alpha_restitution`** (Inelastic Collision Coefficient)

**Location**: `CloningParams(alpha_restitution=...)`

**What it is**: The coefficient of restitution that controls how "elastic" or "inelastic" the velocity collision is during cloning. 

**How it works**: When walker $i$ clones companion $j$, their velocities undergo an inelastic collision:

1. Compute center-of-mass velocity: $v_{\text{cm}} = (v_i + v_j) / 2$
2. Compute relative velocity: $\Delta v = v_j - v_i$
3. Rotate $\Delta v$ randomly: $\Delta v' = R(\Delta v)$ where $R$ is random rotation
4. Apply restitution: 
   - $v_i \leftarrow v_{\text{cm}} - \frac{\alpha}{2} \Delta v'$
   - $v_j \leftarrow v_{\text{cm}} + \frac{\alpha}{2} \Delta v'$

**Effect on velocities**:
- **$\alpha = 0$ (fully inelastic)**: Both walkers get $v_{\text{cm}}$ → velocities become identical → NO velocity diversity
- **$\alpha = 0.5$ (partially inelastic)**: Velocities move toward each other but maintain some difference
- **$\alpha = 1$ (perfectly elastic)**: Relative velocity magnitude preserved (but direction randomized)

**Effect on algorithmic space**:
- **Low values (0.0 - 0.3)**: Velocities collapse quickly → small $\|v_i - v_j\|$ → walkers become close in algorithmic space → clustering
- **High values (0.7 - 1.0)**: Velocities stay diverse → large $\|v_i - v_j\|$ → walkers stay far in algorithmic space → spreading

**Visual effect**: With high $\lambda_{\text{alg}}$ AND low $\alpha_{\text{restitution}}$, the swarm collapses into tight clusters because everyone has similar velocities. High $\alpha$ preserves velocity diversity.

---

### Interactive Comparison: Spreading vs Clustering

Let's run two experiments with extreme parameter settings to see the difference!